In [1]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch_optimizer as optim


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
import torchvision
warnings.filterwarnings('ignore')

from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")
set_global("dataloader_workers", 4)

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook, set_device
get_global("cache_dir")
from facebook_hateful_memes_detector.models import Fasttext1DCNNModel, MultiImageMultiTextAttentionEarlyFusionModel, LangFeaturesModel, AlbertClassifer
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut, ImageAugment, DefinedAffine
from facebook_hateful_memes_detector.training import *
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)


https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/11

# Params

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)


choice_probas = {"keyboard": 0.1, "char_substitute": 0.0, "char_insert": 0.1, "char_swap": 0.0, "ocr": 0.0, "char_delete": 0.1,
                 "fasttext": 0.0, "glove_twitter": 0.0, "glove_wiki": 0.0, "word2vec": 0.0, "split": 0.1,
                 "stopword_insert": 0.3, "word_join": 0.1, "word_cutout": 0.8,
                 "text_rotate": 0.5, "sentence_shuffle": 0.5, "one_third_cut": 0.4, "half_cut":0.1}
preprocess_text = TextAugment([0.0, 0.1, 0.05, 0.35, 0.3, 0.2], choice_probas, fasttext_file="wiki-news-300d-1M-subword.bin")

augs_dict = dict(
    grayscale=transforms.Grayscale(num_output_channels=3),
    hflip=transforms.RandomHorizontalFlip(p=1.0),
    rc2=transforms.Compose(
        [transforms.Resize(480),
         transforms.CenterCrop(400)]),
    rotate=DefinedRotation(15),
    affine=DefinedAffine(0, scale=(0.6, 0.6)),
    translate1=DefinedAffine(0, translate=(0.25, 0.25)),
)
im_transform = ImageAugment(count_proba=[0.0, 1.0], augs_dict=augs_dict, choice_probas="uniform")



data = get_datasets(data_dir="../data/", train_text_transform=preprocess_text, train_image_transform=im_transform, 
                    test_text_transform=None, test_image_transform=None, 
                    cache_images = True, use_images = True, dev=False, 
                    keep_original_text=False, keep_original_image=False, 
                    keep_processed_image=True, keep_torchvision_image=True,)



In [3]:
sgd = torch.optim.SGD
sgd_params = dict(lr=2e-2, momentum=0.9, dampening=0, weight_decay=0, nesterov=False)

rangerQH = optim.RangerQH
rangerQHparams = dict(lr=1e-3, betas=(0.9, 0.999), nus=(.7, 1.0),
    weight_decay=0.0,
    k=6,
    alpha=.5,
    decouple_weight_decay=True,
    eps=1e-8,)

adam = torch.optim.Adam
adam_params = params=dict(lr=1e-4, weight_decay=1e-6)

adamw = torch.optim.AdamW
adamw_params = dict(lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-2)

novograd = optim.NovoGrad
novograd_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
    grad_averaging=False,
    amsgrad=False,)

qhadam = optim.QHAdam
qhadam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,)

radam = optim.RAdam
radam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,)

yogi = optim.Yogi
yogi_params = dict(lr= 1e-2,
    betas=(0.9, 0.999),
    eps=1e-3,
    initial_accumulator=1e-6,
    weight_decay=0)




In [4]:
batch_size=256
epochs = 25
optimizer = adam
optimizer_params = adam_params

scheduler_init_fn = get_multistep_lr([5, 7, 10, 17], gamma=0.1) # get_cosine_schedule_with_warmup # get_cosine_with_hard_restarts_schedule_with_warmup
scheduler_init_fn = get_cosine_schedule_with_warmup()
reg_sched = get_regularizer_scheduler()

lr_strategy = {"im_models": {"torchvision_resnet18": {"7": {"lr": optimizer_params["lr"]/750},
                                                      "8": {"lr": optimizer_params["lr"]/250},
                                                      "lr": optimizer_params["lr"]/2000},
                             "torchvision_resnet50_swsl": {"7": {"lr": optimizer_params["lr"]/1000},
                                                           "8": {"lr": optimizer_params["lr"]/500},
                                                           "lr": optimizer_params["lr"]/4000},
                            "lr": optimizer_params["lr"]/4000},
              }

# {"lr": optimizer_params["lr"]/500}



# ImageAttentionModel

In [ ]:
# {"model": "torchvision_resnet18", "large_rf": True, "finetune": True}
        
lr_strategy = {"im_models": {"torchvision_resnet18_ssl": {"7": {"lr": optimizer_params["lr"]/200},
                                                           "8": {"lr": optimizer_params["lr"]/100},
                                                           "lr": optimizer_params["lr"]/1000},
                            "lr": optimizer_params["lr"]/1000},
              }
model_fn = model_builder(MultiImageMultiTextAttentionEarlyFusionModel, 
                         dict(image_models=[{"model": "torchvision_resnet18_ssl", "large_rf": True, "finetune": True}], 
                              num_classes=2, 
                              text_models=[dict(cls=Fasttext1DCNNModel, params=dict(classifier_dims=256, 
                                                                                    num_classes=2, n_tokens_out=16,
                                                                                    n_layers=2, final_layer_builder=lambda *args: None,
                                                                                    gaussian_noise=0.2, dropout=0.2, 
                                                                                    embedding_dims=256, internal_dims=256, featurizer="cnn",),
                                               in_channels=256, in_tokens=64, forward="get_word_vectors")],
                              internal_dims=256, classifier_dims=256,
                              n_tokens_out=16, n_layers=2,
                              final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.2, dropout=0.2, ),
                         per_param_opts_fn=lr_strategy,
                         optimiser_class=optimizer, optimiser_params=optimizer_params)



In [ ]:
data["metadata"]["use_images"] = True

multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[2, 4, 7, 9, 11, 14, 17, 20, 23])
r1, p1 = results, prfs
results
prfs

# 53m 12s 16061504 0.761	0.733 -> resnet18
# 53m 10s 16061504 0.751	0.740 -> resnet18_swsl
# 53m 12s 16061504 0.757	0.745 -> resnet18_ssl
# 53m 19s 16061504 0.783	0.727 -> resnet18_swsl
# 57m 20s 26240576 0.853	0.687 -> resnet50_swsl
# 59m 39s 25677632 0.820	0.709 -> resnext50_32x4d_swsl
# 1h 21m 0.741	0.717 -> resnext50_32x4d_swsl
# resnext50_32x4d_swsl resnext50_32x4d resnext101_32x8d_wsl resnet50_swsl resnet50_ssl resnext50_32x4d_ssl


# Image Only

In [ ]:
# {"model": "torchvision_resnet18", "large_rf": True, "finetune": True}
        
lr_strategy = {"im_models": {"torchvision_resnet18_ssl": {"7": {"lr": optimizer_params["lr"]/200},
                                                           "8": {"lr": optimizer_params["lr"]/100},
                                                           "lr": optimizer_params["lr"]/1000},
                            "lr": optimizer_params["lr"]/1000},
              }
model_fn = model_builder(MultiImageMultiTextAttentionEarlyFusionModel, 
                         dict(image_models=[{"model": "torchvision_resnet18_ssl", "large_rf": True, "finetune": True}], 
                              num_classes=2, 
                              text_models=[],
                              internal_dims=256, classifier_dims=256,
                              n_tokens_out=16, n_layers=2,
                              final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.2, dropout=0.2, ),
                         per_param_opts_fn=lr_strategy,
                         optimiser_class=optimizer, optimiser_params=optimizer_params)


data["metadata"]["use_images"] = True

multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[2, 4, 7, 9, 11, 14, 17, 20, 23])
r1, p1 = results, prfs
results
prfs



# Detr Model

In [ ]:
# {"model": "torchvision_resnet18", "large_rf": True, "finetune": True}

model_fn = model_builder(
    MultiImageMultiTextAttentionEarlyFusionModel,
    dict(
        image_models=['detr_resnet50'],
        num_classes=2,
        text_models=[
            dict(cls=Fasttext1DCNNModel,
                 params=dict(
                     classifier_dims=256,
                     num_classes=2,
                     n_tokens_out=16,
                     n_layers=2,
                     final_layer_builder=lambda *args: None,
                     gaussian_noise=0.2,
                     dropout=0.2,
                     embedding_dims=256,
                     internal_dims=256,
                     featurizer="cnn",
                 ),
                 in_channels=256,
                 in_tokens=64,
                 forward="get_word_vectors")
        ],
        internal_dims=256,
        classifier_dims=256,
        n_tokens_out=16,
        n_layers=2,
        final_layer_builder=fb_1d_loss_builder,
        gaussian_noise=0.3,
        dropout=0.3,
    ),
    per_param_opts_fn=None,
    optimiser_class=optimizer,
    optimiser_params=optimizer_params)

data["metadata"]["use_images"] = True

batch_size=128
epochs = 10
multi_eval = False
kfold = False
results, prfs = train_validate_ntimes(
    model_fn,
    data,
    batch_size,
    epochs,
    augmentation_weights=augmentation_weights,
    kfold=kfold,
    multi_eval=multi_eval,
    scheduler_init_fn=scheduler_init_fn,
    validation_epochs=[2, 4, 7, 9, 11, 14, 17, 20, 23])
r1, p1 = results, prfs
results
prfs

# "detr_demo", 'detr_resnet50', 'detr_resnet50_panoptic', 'detr_resnet101', 'detr_resnet101_panoptic', "caption_features"

# Caption Features

In [ ]:
# {"model": "torchvision_resnet18", "large_rf": True, "finetune": True}

model_fn = model_builder(MultiImageMultiTextAttentionEarlyFusionModel, 
                         dict(image_models=['caption_features'], 
                              num_classes=2, 
                              text_models=[dict(cls=Fasttext1DCNNModel, params=dict(classifier_dims=256, 
                                                                                    num_classes=2, n_tokens_out=16,
                                                                                    n_layers=2, final_layer_builder=lambda *args: None,
                                                                                    gaussian_noise=0.2, dropout=0.2, 
                                                                                    embedding_dims=256, internal_dims=256, featurizer="cnn",),
                                               in_channels=256, in_tokens=64, forward="get_word_vectors")],
                              internal_dims=256, classifier_dims=256,
                              n_tokens_out=16, n_layers=2,
                              final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.2, dropout=0.2, ),
                         per_param_opts_fn=None,
                         optimiser_class=optimizer, optimiser_params=optimizer_params)


data["metadata"]["use_images"] = True

multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[2, 4, 7, 9, 11, 14, 17, 20, 23])
r1, p1 = results, prfs
results
prfs

# "detr_demo", 'detr_resnet50', 'detr_resnet50_panoptic', 'detr_resnet101', 'detr_resnet101_panoptic', "caption_features"



# Multi-Text Multi Image

In [ ]:
lr_strategy = {
    "im_models": {
        "lr": optimizer_params["lr"] / 10,
        "torchvision_resnet18_ssl-contrastive": {
            "lambd": {
                "8": {
                    "finetune": True
                }
            },
            "lr": optimizer_params["lr"] / 10,
            "finetune": False,
        },
        "vgg_face": {
            "lr": optimizer_params["lr"] / 10,
            "lambd": {
                "0": {
                    "feat_extract": {
                        "finetune": True
                    }
                }
            },
            "finetune": False,
        }
    }
}

model_fn = model_builder(
    MultiImageMultiTextAttentionEarlyFusionModel,
    dict(
        image_models=[
            #             {
            #                 "model": 'caption_features',
            #                 "gaussian_noise": 0.0,
            #                 "dropout": 0.0
            #             },
            {
                "model": 'vgg_face',
                "gaussian_noise": 0.0,
                "dropout": 0.0,
            },
            #             {
            #                 "model": 'detr_resnet50',
            #                 "gaussian_noise": 0.0,
            #                 "dropout": 0.0
            #             },
            #             {
            #                 "model": 'detr_resnet50_panoptic',
            #                 "gaussian_noise": 0.0,
            #                 "dropout": 0.0
            #             },
            {
                "model": "torchvision_resnet18_ssl-contrastive",
                "large_rf": True,
            },
        ],
        num_classes=2,
        text_models=[
            dict(cls=Fasttext1DCNNModel,
                 params=dict(
                     classifier_dims=256,
                     num_classes=2,
                     n_tokens_in=64,
                     n_tokens_out=16,
                     n_layers=2,
                     final_layer_builder=lambda *args: None,
                     gaussian_noise=0.5,
                     dropout=0.4,
                     embedding_dims=256,
                     internal_dims=256,
                     featurizer="gru",
                 ),
                 in_channels=256,
                 in_tokens=64,
                 forward="get_word_vectors"),
            dict(
                cls=AlbertClassifer,
                params=dict(classifier_dims=256,
                            num_classes=2,
                            embedding_dims=768,
                            gaussian_noise=5.5,
                            dropout=0.175,
                            word_masking_proba=0.25,
                            internal_dims=512,
                            final_layer_builder=fb_1d_loss_builder,
                            n_layers=2,
                            n_encoders=2,
                            n_decoders=2,
                            n_tokens_in=96,
                            n_tokens_out=16,
                            featurizer="transformer",
                            model='./model-nsp',
                            loss="focal",
                            dice_loss_coef=0.0,
                            auc_loss_coef=0.0,
                            finetune=False),
                in_channels=768,
                in_tokens=96,
                forward="get_word_vectors",
            )
        ],
        internal_dims=256,
        classifier_dims=256,
        n_tokens_out=32,
        n_layers=2,
        n_encoders=2,
        n_decoders=2,
        final_layer_builder=fb_1d_loss_builder,
        gaussian_noise=0.75,
        dropout=0.3,
    ),
    per_param_opts_fn=lr_strategy,
    optimiser_class=optimizer,
    optimiser_params=optimizer_params)

data["metadata"]["use_images"] = True
batch_size = 64
epochs = 5
kfold = False
results, prfs = train_validate_ntimes(
    model_fn,
    data,
    batch_size,
    epochs,
    kfold=kfold,
    scheduler_init_fn=scheduler_init_fn,
    validation_epochs=[7, 11, 14, 17, 20, 23, 27],
    show_model_stats=True,
    sampling_policy="without_replacement",
    accumulation_steps=4,
    model_call_back=reg_sched,
)
r1, p1 = results, prfs
results
prfs

# "detr_demo", 'detr_resnet50', 'detr_resnet50_panoptic', 'detr_resnet101', 'detr_resnet101_panoptic', "caption_features"

Loading saved model:  resnet18_ssl-contrastive.pth


Trainable Params = 92430082 
 MultiImageMultiTextAttentionEarlyFusionModel(
  (torchvision_pool): AdaptiveAvgPool2d(output_size=1)
  (im_models): ModuleDict(
    (torchvision_resnet18_ssl-contrastive): LambdaLayer(
      (lambd): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=


Epoch =  1 Loss = 0.246657 LR = 0.00003333



Epoch =  2 Loss = 0.228906 LR = 0.00006667



Epoch =  3 Loss = 0.219330 LR = 0.00010000



Epoch =  4 Loss = 0.210530 LR = 0.00009505



Epoch =  5 Loss = 0.190959 LR = 0.00008117


# Predict

In [ ]:
batch_size=512
epochs = 1

submission, text_model = train_and_predict(model_fn, data, batch_size, epochs, augmentation_weights, scheduler_init_fn=scheduler_init_fn)
submission.to_csv("submission.csv",index=False)
submission.sample(3)


In [ ]:
submission.sample(10)